In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os, sys, shutil, subprocess
import jsonlines
import numpy as np
import pandas as pd
from joblib import Parallel, delayed

In [ ]:
from verilator_coverage_src import *

In [ ]:
# load dataset from Olivia
o100data = load_jsonl("/mnt/shared/gpfs/escad_verilog_dataset/verilator_coverage_jinghua/manual_tb.jsonl")

In [ ]:
# load big dataset from Manvi
o312kdata = load_jsonl("/mnt/shared/gpfs/escad_verilog_dataset/verilator_coverage_jinghua/tb_dataset/with_keys_312k_sep_5.jsonl")

In [ ]:
OVERALL_FOLDER = "/mnt/shared/gpfs/escad_verilog_dataset/verilator_coverage_jinghua/run_results_2025-10-10"

# need to load verilator container image before running "run_verilator"
init_verilator_env()

# run all modules and generate coverage reports
all_res = []
for i in tqdm(range(len(o100data))):
    _tmp_dir = os.path.join(OVERALL_FOLDER, "O100_manual_"+str(i))
    if os.path.isdir(_tmp_dir):
        continue
    _modules_list = [o100data[i]['full_text']]
    _modules_list.extend( find_all_dependency_modules(data_list=o312kdata, starting_index_list=[o100data[i]['index']]) )
    _modules_list = list(set(_modules_list))
    all_res.append( run_verilator(modules_full_code_list=_modules_list, tmp_dir=_tmp_dir,
                                 force_rerun=False, timeout_seconds=120) )

In [ ]:
report_count = 0
for d in sorted(os.listdir(OVERALL_FOLDER), key=lambda x: int(x.split("_")[-1])):
    try:
        res_list = parse_coverage_log(os.path.join(OVERALL_FOLDER, d, "obj_dir/coverage.dat"))
#         print(d)
        print(d, "   ", np.sum(res_list), "    ", np.mean(res_list), "    ", np.max(res_list))
#         print()
        report_count += 1
    except:
        pass

In [ ]:
report_count